# Compare unets with multiple inputs
Vanilla u net  
Unet with 2 inputs  
Unet with 2 inputs on different depth  
Double U net  
Triple U net  

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/M_DV_V2022

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.layers import Input 
from keras.utils.vis_utils import plot_model

from unet_detection.models.vanilla_unet import vanilla_unet
from unet_detection.models.unet_2input import unet_2input
from unet_detection.models.unet_2input_deep import unet_2input_deep
from unet_detection.models.double_unet import double_unet
from unet_detection.models.triple_unet import triple_unet

### All nets

In [ ]:
seed = 123

vUnet = vanilla_unet(seed)
unet2i = unet_2input(seed)
unet2i_d = unet_2input_deep(seed)
dUnet = double_unet(seed)
tUnet = triple_unet(seed)

### Import data set and show

In [ ]:
X_train = np.load("data/u_net_roofs/X_data.npy") 
X_70_train = np.load("data/u_net_roofs/X_70_data.npy")
y_train = np.load("data/u_net_roofs/y_data.npy")

X_val = np.load("data/u_net_roofs/X_data_val.npy")
X_70_val = np.load("data/u_net_roofs/X_70_data_val.npy")
y_val = np.load("data/u_net_roofs/y_data_val.npy")

X_test = np.load("data/u_net_roofs/X_data_test.npy")
X_70_test = np.load("data/u_net_roofs/X_70_data_test.npy")
y_test = np.load("data/u_net_roofs/y_data_test.npy")

In [ ]:
def redesign_y(y):
  n,r1,c1,d = y.shape
  # Adds a new dimension of layer too have two class problem.
  yy = np.append(y, np.zeros((n, r1, c1,d)), axis=3)
  for i in range(int(y.max()-1)):  
    yy = np.append(yy, np.zeros((n, r1, c1,d)), axis=3)
  yy1 = yy.copy()
  yy1[:,:,:,0] = 0 # reset map
  for i in range(n):
    values = yy[i,:,:,0]
    for r in range(r1):
      for c in range(c1):
        value = yy[i,r,c,0]
        yy1[i,r,c,int(value)] = 1

  return yy1

In [ ]:
y_train = redesign_y(y_train)
y_val = redesign_y(y_val)
y_test = redesign_y(y_test)

print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

In [ ]:
# Scale image down so each border is the same size when upsizing!
X_70_train = X_70_train[:,4:-4,4:-4,:]
X_70_val = X_70_val[:,4:-4,4:-4,:]
X_70_test = X_70_test[:,4:-4,4:-4,:]

## Set up net

In [ ]:
input_img1 = Input(shape=(128,128,177))
input_img2 = Input(shape=(56,56,220))

model = dUnet.get_unet(input_img1, input_img2, n_classes=2, last_activation='softmax')

model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

## Run test

In [ ]:
hhh = model.fit([X_train, X_70_train],
               y_train, 
               validation_data=([X_val, X_70_val],y_val), 
               batch_size=16,
               epochs=10)

## Display model net

In [ ]:
#plot_model(model, to_file="doubleUnet.png", dpi=300, rankdir='TB')